In [ ]:
from lightning.pytorch.cli import LightningArgumentParser

In [ ]:
import init
from src.model.clip.snvl import CLIPTextEmbedding
import src.clip as clip
clip.tokenize([f"1 2 3 4 5 6 7 8 9 10", "12313123"]).shape
# m = CLIPTextEmbedding()

In [ ]:
import pickle
with open("./misc/temp.pickle", "rb") as f:
    data = pickle.load(f)

In [ ]:
import torch
points = torch.tensor(data['k']['lips'][0])

In [ ]:
point_dist = torch.nn.functional.mse_loss(
    points.unsqueeze(0), points.unsqueeze(1), reduction="none"
).flatten(1).sum(1)
rev_dis_idx = point_dist.sort(descending=True)[1]

In [ ]:
target_indices = rev_dis_idx[int(rev_dis_idx.shape[0] * 0.5):]

In [ ]:
points[target_indices].mean(0)

In [ ]:

import os
import cv2
import random
import numpy as np

from deepface import DeepFace
from glob import glob
from src.dataset.ffpp import FFPP, FFPPAugmentation, FFPPSampleStrategy
SAVE_FOLDER = "./misc/extern/ffpp_samples/"
NUM_SAMPLES = 100
for df_type in ["REAL", "DF", "FS", "NT", "F2F"]:
    dataset = FFPP(
        df_types=[df_type],
        compressions=["c23"],
        n_px=224,
        strategy=FFPPSampleStrategy.NORMAL,
        augmentations=FFPPAugmentation.NONE,
        force_random_speed=False,
        vid_ext=".avi",
        data_dir="datasets/ffpp/",
        num_frames=1,
        clip_duration=4,
        transform=lambda x: x,
        split="val",
        pack=False,
        ratio=1.0
    )

    random.seed(1234)
    base_dir = os.path.join(SAVE_FOLDER, df_type)
    os.makedirs(base_dir, exist_ok=True)
    # random sample datas
    for i, idx in enumerate([random.randrange(0, len(dataset)) for _ in range(NUM_SAMPLES)]):
        image = dataset.get_entity(idx)["clips"]
        image = image.flatten(0, 2).permute(1, 2, 0).numpy()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #     cv2.imwrite(os.path.join(base_dir, f"{i}.png"), image)
        print(
        )
        break
    break

    # gender_record = {}
    # for file in glob(os.path.join(base_dir, "*.png")):
    #     print(file)
    #     file_name = os.path.splitext(os.path.split(file)[0])[0]
    #     dom_gender = DeepFace.analyze(
    #         img_path=file,
    #         actions=['gender'],
    #         enforce_detection=False
    #     )[0]["dominant_gender"]
    #     gender_record[file_name] = (0 if dom_gender == "Woman" else 1)
    # gender_record_path = os.path.join(base_dir, "gender.pickle")
    # with open(gender_record_path, "wb") as f:
    #     pickle.dump(gender_record, f)

In [ ]:
from deepface import DeepFace

objs = DeepFace.analyze(
    img_path="test.png",
    actions=['gender']
)

In [ ]:
objs

In [ ]:
from src.clip import clip as CLIP
CLIP.tokenize("woman,lady,female,").shape

In [ ]:
import cv2
import torch
import random
from deepface import DeepFace
from notebooks.tools import load_model
from src.clip import clip as CLIP
from src.model.clip import FrameAttrExtractor
from src.clip.model_vpt import PromptMode, PromptMask
from src.dataset.ffpp import FFPP, FFPPSampleStrategy, FFPPAugmentation
DEVICE = "cuda"
NUM_SAMPLES = 100
model, transform = CLIP.load("ViT-B/16")
model = model.float()

dataset = FFPP(
    df_types=["REAL"],
    compressions=["c23"],
    n_px=224,
    strategy=FFPPSampleStrategy.NORMAL,
    augmentations=FFPPAugmentation.NONE,
    force_random_speed=False,
    vid_ext=".avi",
    data_dir="datasets/ffpp/",
    num_frames=1,
    clip_duration=4,
    split="val",
    transform=transform,
    pack=False,
    ratio=1.0
)
random.seed(1234)

# random sample datas
clips = []
for idx in [random.randrange(0, len(dataset)) for _ in range(NUM_SAMPLES)]:
    clips.append(dataset.get_entity(idx)["clips"])
clips.append(dataset.get_entity(38)["clips"])
clips = torch.cat(clips, dim=0).to(DEVICE)

# detect gender
gender_clip_idx_map = {
    "Woman": [],
    "Man": []
}
for clip_idx in range(clips.shape[0]):
    image = clips[[clip_idx]].cpu()
    image = image.flatten(0, 2).permute(1, 2, 0).numpy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gender = DeepFace.analyze(
        img_path=image,
        actions=['gender'],
        enforce_detection=False,
        silent=True
    )[0]["dominant_gender"]
    gender_clip_idx_map[gender].append(clip_idx)

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(2 * clips.shape[0], 2))
images = clips.squeeze(1).permute(2, 0, 3, 1).flatten(1, 2).cpu().numpy()
images = (images - images.min()) / (images.max() - images.min())
plt.imshow(images)

In [ ]:
gender_clip_idx_map

In [ ]:
from sklearn.metrics import accuracy_score
with torch.no_grad():
    text = CLIP.tokenize(["a man", "a woman"]).to(DEVICE)
    clips = clips.squeeze(1)
    logits_per_image, logits_per_text = model(clips, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    label = [1 if i in gender_clip_idx_map["Woman"] else 0 for i in range(clips.shape[0])]
    pred = probs[:, 1] > 0.5
accuracy_score(label, pred)

In [ ]:
probs[-1]

In [ ]:
import numpy as np
from sklearn.manifold import TSNE


def tsne(
    embeddings,
    labels,
    perplexities=[10, 50, 80],
    graph_title="",
    save_path="",
    save=False
):
    X = np.array(embeddings)
    plt.figure(figsize=(5 * len(perplexities), 4), layout="constrained")
    plt.suptitle(graph_title)
    for i, perplexity in enumerate(perplexities):
        print(f"TSNE Running(Perplexity:{perplexity})....")
        tsne = TSNE(
            n_components=2,
            n_iter=10000,
            learning_rate='auto',
            init='random',
            perplexity=perplexity,
            # metric="cosine"
        )
        _X = tsne.fit_transform(X)
        print(f"TSNE Completed.")

        color = {
            "REAL": "green",
            "DF": "blue",
            "FS": "purple",
            "F2F": "darkorange",
            "NT": "red",
            "CDF_REAL": "turquoise",
            "CDF_FAKE": "deeppink",
            "DFDC_REAL": "forestgreen",
            "DFDC_FAKE": "steelblue",
            "Woman": "red",
            "Man": "blue"
        }

        plt.subplot(1, len(perplexities), i + 1)
        plt.title(f"Perplexity:{perplexity}")
        plt.gca().set_xticks([])
        plt.gca().set_yticks([])

        offset = 0
        for category_type, num in labels:
            plt.scatter(
                _X[offset:offset + num, 0],
                _X[offset:offset + num, 1],
                3,
                # color="green" if category_type == "REAL" else "red",
                color=color[category_type],
                label=category_type if i == 0 else "",
                alpha=0.5
            )
            offset += num

        assert offset == len(_X)

    plt.gcf().legend(loc='outside right center')
    if (save):
        folder, file = os.path.split(save_path)
        if (not os.path.exists(folder)):
            os.makedirs(folder, exist_ok=True)
        plt.savefig(save_path)
    else:
        plt.show()
    plt.close()

In [ ]:
import numpy as np
with torch.no_grad():
    embeds = model.encode_image(clips)
    tsne_embeddings = []
    tsne_labels = []
    for gender in gender_clip_idx_map:
        gender_embeds = embeds[gender_clip_idx_map[gender]].tolist()
        tsne_embeddings.extend(gender_embeds)
        tsne_labels.append((gender, len(gender_embeds)))
    tsne(tsne_embeddings, tsne_labels, [1, 5, 10, 50, 70])

In [ ]:
dataset.video_list

In [ ]:
video_idx = next(i for i, data in enumerate(dataset.video_list) if data[2] == "991")
entity_idx = dataset.stack_video_clips[video_idx - 1]
clips = dataset.get_entity(entity_idx)["clips"]

In [ ]:
# image = clips[0, 0].permute(1, 2, 0)
# print(image.min(), image.max())
# image = (image - image.min()) / (image.max() - image.min())
# plt.imshow(image)

In [ ]:
# import torch
# import clip
# from PIL import Image
# image = Image.open("man.png")
# image = preprocess(image).permute(1, 2, 0)
# print(image.min(), image.max())
# image = (image - image.min()) / (image.max() - image.min())
# plt.imshow(image)

In [1]:
import torch
# import clip as CLIP
from src.clip import clip as CLIP
from PIL import Image
device = "cuda" if torch.cuda.is_available() else "cpu"

model, preprocess = CLIP.load("ViT-B/16", device=device)
model = model.float()

In [2]:
image = Image.open("man.png")
image = preprocess(image).unsqueeze(0).to(device)

In [3]:
text = CLIP.tokenize(["a man", "a woman"]).to(device)

In [4]:

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[0.99109334 0.00890664]]


In [5]:
from src.model.clip import FrameAttrExtractor
from src.clip.model_vpt import PromptMode
image_embed = FrameAttrExtractor("ViT-B/16", PromptMode.NONE, 0, 0, 0, True)
image_embed = image_embed.to(device)

In [6]:
# import torch.nn as nn


# class TextAffinityHead(nn.Module):
#     def __init__(
#         self,
#         out_dim=2,
#         n_prompts=10,
#         architecture: str = "ViT-B/16",
#         gender_text=["a man", "a woman"]
#     ):
#         super().__init__()
#         model, _ = CLIP.load(architecture, "cpu")
#         model.visual = None
#         model = model.float().requires_grad_(False)
#         self.out_dim = out_dim
#         self.n_prompts = n_prompts
#         self.ln_final = model.ln_final
#         self.n_ctx = model.context_length
#         self.logit_scale = model.logit_scale
#         self.transformer = model.transformer
#         self.text_projection = model.text_projection
#         self.token_embedding = model.token_embedding
#         self.positional_embedding = model.positional_embedding
#         self.gender_text = gender_text

#         # Inversion
#         text_tokens = CLIP.tokenize(gender_text)
#         text_embeds = self.token_embedding(text_tokens)
#         indices = text_tokens.max(dim=1)[1]
#         assert not False in (indices == indices[0])
#         text_embeds = text_embeds[:, 1:indices[0]]
#         self.gender_token = nn.Parameter(
#             text_embeds.repeat_interleave(self.out_dim, dim=0),
#             requires_grad=False
#         )
#         # generic token and prompts
#         tokens = self.token_embedding(CLIP.tokenize(""))[0]
#         self.beg_token = nn.Parameter(
#             tokens[0].unsqueeze(0).expand(
#                 len(self.gender_text) * self.out_dim,
#                 -1,
#                 -1
#             ),
#             requires_grad=False
#         )
#         self.end_token = nn.Parameter(
#             tokens[1].unsqueeze(0).expand(
#                 len(self.gender_text) * self.out_dim,
#                 -1,
#                 -1
#             ),
#             requires_grad=False
#         )
#         self.null_token = nn.Parameter(
#             tokens[2].unsqueeze(0).expand(
#                 len(self.gender_text) * self.out_dim,
#                 self.n_ctx - self.n_prompts - 2 - self.gender_token.shape[1],
#                 -1
#             ),
#             requires_grad=False
#         )
#         self.cls_text_embed = nn.Parameter(
#             (tokens.shape[1] ** -0.5) * torch.randn(
#                 len(self.gender_text) * self.out_dim,
#                 self.n_prompts,
#                 tokens.shape[1]
#             ),
#             requires_grad=True
#         )

#     def create_anchors(self):
#         x = torch.cat(
#             (
#                 self.beg_token,
#                 self.gender_token,
#                 self.cls_text_embed,
#                 self.end_token,
#                 self.null_token
#             ),
#             dim=1
#         )  # [batch_size, n_ctx, d_model]
#         # assert not False in (x[0] == x[1])
#         x = x + self.positional_embedding
#         x = x.permute(1, 0, 2)  # NLD -> LND
#         x = self.transformer(x)
#         x = x.permute(1, 0, 2)  # LND -> NLD
#         x = self.ln_final(x)

#         # x.shape = [batch_size, n_ctx, transformer.width]
#         # take features from the eot embedding (eot_token is the highest number in each sequence)
#         x = x[
#             :,
#             (self.gender_token.shape[1] + 1 + self.n_prompts)
#         ] @ self.text_projection

#         return x

#     def forward(self, features):
#         anchors = self.create_anchors()

#         # normalized features
#         features = features / features.norm(dim=1, keepdim=True)
#         anchors = anchors / anchors.norm(dim=1, keepdim=True)

#         # cosine similarity as logits
#         logit_scale = self.logit_scale.exp()
#         logits = logit_scale * features @ anchors.t()
#         # calculate the gender averaged probability
#         logits = logits.view(-1, len(self.gender_text), self.out_dim).mean(dim=-1)
#         return logits


# text_embed = TextAffinityHead(n_prompts=1, out_dim=2)
# text_embed = text_embed.to(device)
# # data = text_embed.create_anchors()
# # (data[3] - text_features[1]).abs() < 1e-5

In [12]:
with torch.no_grad():
    _image_features = image_embed(image)["embeds"]
    logits = text_embed(_image_features, mean_mode="none")

    probs = logits.softmax(dim=-1).cpu().numpy()
probs
# print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

array([[0.40874672, 0.5224272 , 0.06592832, 0.00289775]], dtype=float32)